# Add items for human feedback

In [2]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [3]:
import io
import json
import uuid
import time
import boto3
import botocore

# Amazon Python SDK clients
sagemaker = boto3.client("sagemaker", region)
comprehend = boto3.client("comprehend", region)
a2i = boto3.client("sagemaker-a2i-runtime")
s3 = boto3.client("s3", region)

# Retrieve the `augmented_ai_flow_definition_arn` Created Previously

In [4]:
%store -r augmented_ai_flow_definition_arn

In [5]:
print(augmented_ai_flow_definition_arn)

arn:aws:sagemaker:us-east-1:079002598131:flow-definition/fd-dsoaws-star-rating-c2ceaad3-75f7-4321-b37b-012d5d5f9754


In [8]:
items = ["I enjoy this product", 
        "I am unhappy with this product", 
        "It is okay", 
        "Sometimes it works"]

In [9]:
human_loops_started = []

for item in items:

    print(f'Processing item: "{item}"')

    humanLoopName = str(uuid.uuid4())
    inputContent = {"initialValue": 3, "taskObject": item}
    start_loop_response = a2i.start_human_loop(
        HumanLoopName=humanLoopName,
        FlowDefinitionArn=augmented_ai_flow_definition_arn,
        HumanLoopInput={"InputContent": json.dumps(inputContent)},
    )

    human_loops_started.append(humanLoopName)

    print(f"*** ==> Starting human loop with name: {humanLoopName}  \n")

Processing item: "I enjoy this product"
*** ==> Starting human loop with name: 45c73325-6b2c-4d2c-be30-e30784e50e74  

Processing item: "I am unhappy with this product"
*** ==> Starting human loop with name: d2d7312d-3f8a-458c-a234-7965814a7d75  

Processing item: "It is okay"
*** ==> Starting human loop with name: c3170a8b-74e8-4d95-9cdc-1fa3d8c7c228  

Processing item: "Sometimes it works"
*** ==> Starting human loop with name: 068b23b1-0396-47a4-9c0f-d0fa605425df  



# Check Status of Human Loop

In [10]:
completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f"HumanLoop Name: {human_loop_name}")
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print("")

    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)

HumanLoop Name: 45c73325-6b2c-4d2c-be30-e30784e50e74
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-079002598131/a2i-star-rating-results/fd-dsoaws-star-rating-c2ceaad3-75f7-4321-b37b-012d5d5f9754/2023/03/20/18/22/27/45c73325-6b2c-4d2c-be30-e30784e50e74/output.json'}

HumanLoop Name: d2d7312d-3f8a-458c-a234-7965814a7d75
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-079002598131/a2i-star-rating-results/fd-dsoaws-star-rating-c2ceaad3-75f7-4321-b37b-012d5d5f9754/2023/03/20/18/22/27/d2d7312d-3f8a-458c-a234-7965814a7d75/output.json'}

HumanLoop Name: c3170a8b-74e8-4d95-9cdc-1fa3d8c7c228
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-079002598131/a2i-star-rating-results/fd-dsoaws-star-rating-c2ceaad3-75f7-4321-b37b-012d5d5f9754/2023/03/20/18/22/27/c3170a8b-74e8-4d95-9cdc-1fa3d8c7c228/output.json'}

HumanLoop Name: 068b23b1-0396-47a4

# Wait For Workers to Complete Their Human Loop Tasks

Navigate to the link below and login with your email and password that you used when you set up the Private Workforce.

In [11]:
%store -r augmented_ai_workteam_arn

In [12]:
print(augmented_ai_workteam_arn)

arn:aws:sagemaker:us-east-1:079002598131:workteam/private-crowd/dsoaws


In [13]:
workteam_name = augmented_ai_workteam_arn[augmented_ai_workteam_arn.rfind("/") + 1 :]
print(workteam_name)
print("Navigate to the private worker portal and complete the human loop.")
print("Make sure you have invited yourself to the workteam and received the signup email.")
print("Note:  Check your spam filter if you have not received the email.")
print("")
print("https://" + sagemaker.describe_workteam(WorkteamName=workteam_name)["Workteam"]["SubDomain"])

dsoaws
Navigate to the private worker portal and complete the human loop.
Make sure you have invited yourself to the workteam and received the signup email.
Note:  Check your spam filter if you have not received the email.

https://d0xvtcio9i.labeling.us-east-1.sagemaker.aws


# Start Labeling

<img src="img/augmented-comprehend-custom-start-working.png" width="80%" align="left">

# Select Label

<img src="img/augmented-comprehend-custom-select-label.png" width="80%" align="left">

# Loop is Completed

<img src="img/augmented-comprehend-custom-finished-task.png" width="80%" align="left">

In [14]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>

<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}
</script>